<a href="https://colab.research.google.com/github/Zahra-FallahMMA/DRL_Offload_Allocation/blob/main/Offload_Location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import Libraries

In [1]:
import xml.etree.ElementTree as ET
from io import StringIO
import os
import re
import numpy as np
from collections import deque, defaultdict
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.models import load_model
from itertools import product
import tensorflow as tf
!pip install simpy
import simpy

# Set TensorFlow logging level to suppress detailed logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# Class Random Agent

In [2]:
class RandomAgent:

   def __init__(self, action_size):
      self.action_size = action_size

   def choose_action(self):
        return random.randrange(self.action_size)

# class Task

In [3]:
class Task:
    def __init__(self, id, instructions, workflow_id):
        self.id = id
        self.instructions = instructions  # Execution time or computational instructions
        self.children = []  # List of tasks that depend on this task
        self.parents = []  # List of tasks this task depends on

        self.input_files = []   # list of (filename, size)
        self.output_files = []  # list of (filename, size)

        self.executed = False  # Status of execution
        self.executed_on = None  # Node this task was executed on
        self.execution_time = 0  # Time taken to execute the task
        self.cost = 0  # Cost of executing the task
        self.comm_delay = 0  # Communication delay in seconds
        self.workflow_id = workflow_id  # Workflow identifier to which this task belongs
        self.total_transfer_files_size = 0
        self.waiting_time = 0
        self.total_file_size = 0


# Class Workflow

In [4]:
class Workflow:
    def __init__(self, id):
        self.id = id  # Workflow identifier
        self.tasks = {}  # Dictionary of tasks in the workflow

    def add_task(self, task_id, instructions, parent_ids=[], input_files=[], output_files=[]):
        if task_id not in self.tasks:
            self.tasks[task_id] = Task(task_id, instructions, self.id)
        task = self.tasks[task_id]
        for parent_id in parent_ids:
            if parent_id not in self.tasks:
                self.tasks[parent_id] = Task(parent_id, 0, self.id)
            parent_task = self.tasks[parent_id]
            parent_task.children.append(task)
            task.parents.append(parent_task)

        task.input_files.extend(input_files)
        task.output_files.extend(output_files)

# class parse_dax

In [5]:
def parse_dax(file_path, workflow_id):
    tree = ET.parse(file_path)
    root = tree.getroot()

    workflow_id = workflow_id
    workflow = Workflow(workflow_id)

    # Pegasus DAX namespace
    dax_ns = '{http://pegasus.isi.edu/schema/DAX}'

    # Parse jobs
    jobs = {job.attrib['id']: job for job in root.findall(f'{dax_ns}job')}

    # Add jobs to workflow
    for job_id, job_elem in jobs.items():
        instructions = float(job_elem.attrib.get('runtime', 0))


        # NEW: Gather input/output files from <uses link="input" ...> or <uses link="output" ...>
        input_files = []
        output_files = []
        for uses_elem in job_elem.findall(f'.//{dax_ns}uses'):
            link_type = uses_elem.attrib.get('link', '')
            file_name = uses_elem.attrib.get('file', '')
            file_size_str = uses_elem.attrib.get('size', '0')
            file_size = int(file_size_str)

            if link_type == 'input':
                input_files.append((file_name, file_size))  # e.g. ("FFI_0_2_subfx.sgt", 262360916)
            elif link_type == 'output':
                output_files.append((file_name, file_size))

        workflow.add_task(job_id,
                          instructions,
                          parent_ids=[],  # We'll handle parents in a moment,
                          input_files=input_files,
                          output_files=output_files)

    # Parse dependencies
    for child in root.findall(f'{dax_ns}child'):
        child_id = child.attrib['ref']
        parent_ids = [parent.attrib['ref'] for parent in child.findall(f'{dax_ns}parent')]
        workflow.add_task(child_id, 0, parent_ids)  # Adds a child node with its parent nodes, setting instructions to 0 to avoid overwrite

    return workflow


def ensemble_of_workflows(name, size=10, distribution='constant', dax_path=''):
    ws = []
    ensemble = []
    directory_path = dax_path  # Directory containing DAX files

    # List and filter files in directory
    files = os.listdir(directory_path)
    filtered_files = [file for file in files if name in file]

    if distribution == 'constant':
        pattern = r'100(?!\d)'
        for s in filtered_files:
            if re.search(pattern, s):
                ensemble = [s] * size  # Replicate the matched file 'size' times
                break
    else:
        numbers = np.random.randint(0, len(filtered_files), size)
        #print('numbers: ', numbers)
        ensemble = [filtered_files[i] for i in numbers]  # Select random files based on uniform distribution

    w_id = 0
    for name in ensemble:
        ws.append(parse_dax(dax_path+name,w_id))
        w_id = w_id + 1

    return ws

# Loading dax files

In [6]:
from google.colab import drive
import glob

drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/Zahra/dax/'

Mounted at /content/drive


# class Device

In [7]:
class Device:
    def __init__(self, id, mips, cost_per_hour, env):
        self.id = id
        self.mips = mips
        self.cost_per_hour = cost_per_hour
        self.queue = deque()
        self.runnig_queue = deque()
        self.resource = simpy.Resource(env, capacity=1)
        self.dev_type = id.split('_')[0]

    def add_task_to_queue(self, task):
        self.queue.append(task)

    def get_next_task(self):
        return self.queue.popleft() if self.queue else None

    def waiting_time(self):
        waiting_time = 0
        for t in self.queue:
            waiting_time += t.instructions / self.mips
        return waiting_time


# class FogEnv

In [8]:
import simpy

class FogEnv:
    def __init__(self, env, iot_devices, fog_nodes, cloud_servers, workflows):
        self.env = env
        self.iot_devices = iot_devices
        self.fog_nodes = fog_nodes
        self.cloud_servers = cloud_servers
        self.cost = 0
        self.completed_workflows = 0
        self.workflows = workflows
        self.total_delay = 0

        # NEW: Track file origins
        self.file_origins = {}  # { file_name: device_id }

        # Example bandwidth table (Mb/s)
        self.bandwidth_matrix = {
            ('iot', 'iot'): 2000,
            ('iot', 'fog'): 500,
            ('iot', 'cloud'): 500,
            ('fog', 'iot'): 500,
            ('fog', 'fog'): 4000,
            ('fog', 'cloud'): 1000,
            ('cloud', 'iot'): 500,
            ('cloud', 'fog'): 1000,
            ('cloud', 'cloud'): 1000
        }


        # Storage cost rate per GB per hour for each device type (in dollars)
        self.storage_cost_rate = {
            'iot': 0.0007,  # $ per GB per hour for IoT devices
            'fog': 0.00042,  # $ per GB per hour for Fog nodes
            'cloud': 0.00003  # $ per GB per hour for Cloud servers
        }


    def assign_task(self, task, device):
        with device.resource.request() as req:
            yield req

            # Set the arrival time when the task is added to the queue
            task.arrival_time = self.env.now

            # 1) Transfer each input file if needed
            for file_name, file_size_bytes in task.input_files:
                yield self.env.process(self.handle_file_transfer(task, file_name, file_size_bytes, device))


            # 2) Execute the task on this device
            execution_time = task.instructions * 6000/ device.mips
            self.total_delay += execution_time
            task.execution_time = execution_time
            yield self.env.timeout(execution_time)

            # 3) Calculate waiting time (difference between current time and arrival time)
            waiting_time = self.env.now - task.arrival_time

            # 4) Update cost
            task.cost = execution_time * device.cost_per_hour
            self.cost += task.cost

            # 5) Calculate and add storage cost (including waiting time)
            storage_cost = self.calculate_storage_cost(task, device, waiting_time, execution_time)
            task.storage_cost = storage_cost
            #self.cost += storage_cost

            task.executed = True
            task.executed_on = device.id
            task.execution_time = execution_time

            # 6) Mark output files as originating from this device
            for out_file, _sz in task.output_files:
                self.file_origins[out_file] = device.id

            # 7) Check if this workflow is now complete
            wf = next(w for w in self.workflows if w.id == task.workflow_id)
            self.check_workflow_completion(wf)
            device.queue.popleft()


    def calculate_storage_cost(self, task, device, waiting_time, execution_time):
        total_file_size = 0

        # Add the size of input files
        for file_name, file_size_bytes in task.input_files:
            total_file_size += file_size_bytes

        # Add the size of output files
        for out_file, file_size_bytes in task.output_files:
            total_file_size += file_size_bytes


        # Convert total size to GB
        total_file_size_gb = total_file_size / (1024 ** 3)  # Convert bytes to GB

        # Get the storage cost rate for the device type
        device_type = device.dev_type
        storage_cost_per_hour = self.storage_cost_rate.get(device_type, 0)

        # Calculate the total storage time (waiting time + execution time)
        total_storage_time = waiting_time + execution_time

        # Convert total time from seconds to hours
        total_time_in_hours = total_storage_time / 3600  # Convert seconds to hours


        # Calculate the storage cost for this task (based on total storage time)
        storage_cost = total_file_size_gb * storage_cost_per_hour * total_storage_time  # cost per GB per hour * time
        return storage_cost


    # Function to handle transfer
    def handle_file_transfer(self,task, file_name, file_size_bytes, destination_device):


        # If we don't know the file origin, assume it's "local" or from some default place
        if file_name not in self.file_origins:
            return  # no transfer needed

        source_dev_id = self.file_origins[file_name]
        if source_dev_id == destination_device.id:
            return  # already local

        # Get device types for bandwidth lookup
        source_dev = self.get_device_by_id(source_dev_id)
        if source_dev is None:
            return  # fallback: no known device

        task.total_transfer_files_size += file_size_bytes

        source_type = source_dev.dev_type
        dest_type = destination_device.dev_type

        # bandwidth_matrix gives us Mb/s. Convert bytes -> megabits
        # 1 byte = 8 bits, so:
        file_megabits = file_size_bytes * 8 / 1e6

        # Look up or fallback
        band_mbps = self.bandwidth_matrix.get((source_type, dest_type), 1000.0)

        # Transfer time (seconds)
        transfer_time = file_megabits / band_mbps

        # Optional: add base communication latency
        comm_latency = 0.01  # 10 ms, for example
        total_delay = comm_latency + transfer_time
        self.total_delay += total_delay
        task.comm_delay = total_delay



        # Wait out the transfer in simulation time
        yield self.env.timeout(total_delay)

    def get_device_by_id(self, device_id):
        for device in self.iot_devices + self.fog_nodes + self.cloud_servers:
            if device.id == device_id:
                return device
        return None

    def check_workflow_completion(self, workflow):
        # Check if all tasks in the workflow are executed
        if all(task.executed for task in workflow.tasks.values()):
            self.completed_workflows += 1  # Increment completed workflows counter
            # print(f"Workflow {workflow.id} is completed! Total completed workflows: {self.completed_workflows}")





def process_workflow(env, workflow, fog_env, agent):
    # A list to hold all the task processes (concurrent tasks)

    # Iterate through each task in the workflow
    for task in workflow.tasks.values():
        if task.executed:
            continue

        processes = []

        # Check if the task can be executed (all parent tasks are done)
        if all([parent.executed for parent in task.parents]) or task.parents == []:
            # Get all tasks from workflow that are ready to execute (parents done)
              ready_tasks = agent.get_ready_tasks(workflow.tasks.values())

              # Use Offload-Location heuristic to decide allocation of ready tasks
              allocations = agent.offload_location(ready_tasks)
              #print(allocations)


              # For each (task, device) pair, assign the task to the device and start its process
              for task, device in allocations:
                  device.add_task_to_queue(task)
                  task.executed_on = device.id

                  # Create a process for this task assignment and track i
                  process = env.process(fog_env.assign_task(task, device))
                  agent.file_origins = fog_env.file_origins
                  processes.append(process)

    # After all tasks are started, wait for all of them to finish
    yield env.all_of(processes)

In [9]:
import random
from itertools import combinations

class HeuristicAgent:
    def __init__(self, iot_devices, fog_nodes, cloud_servers):
        self.iot_devices = iot_devices
        self.fog_nodes = fog_nodes
        self.cloud_servers = cloud_servers
        self.all_devices = iot_devices + fog_nodes + cloud_servers
        self.storage_cost_rate = {'iot': 0.0007, 'fog': 0.00042, 'cloud': 0.00003}
        self.bandwidth_matrix = {
            ('iot', 'iot'): 2000, ('iot', 'fog'): 500, ('iot', 'cloud'): 500,
            ('fog', 'iot'): 500, ('fog', 'fog'): 4000, ('fog', 'cloud'): 1000,
            ('cloud', 'iot'): 500, ('cloud', 'fog'): 1000, ('cloud', 'cloud'): 1000
        }
        self.file_origins = {}  # file_name -> device_id


    def get_ready_tasks(self, all_tasks):
        return [t for t in all_tasks if not t.executed and (not t.parents or all(p.executed for p in t.parents))]

    def estimate_time(self, task, device):

        total_delay = 0
        for file_name, file_size in task.input_files:
            source_dev_id = self.file_origins.get(file_name, device.id)
            if source_dev_id == device.id:
                continue
            source_dev = self.get_device_by_id(source_dev_id)
            if not source_dev:
                continue
            bandwidth = self.bandwidth_matrix.get((source_dev.dev_type, device.dev_type), 1000)
            file_megabits = file_size * 8 / 1e6
            transfer_time = file_megabits / bandwidth
            total_delay += transfer_time + 0.01  # Add latency
        if device.mips <= 0:
            raise ValueError(f"Device {device.id} has invalid MIPS.")
        execution_time = task.instructions * 6000 / device.mips
        return total_delay + execution_time

    def estimate_cost(self, task, device):
        total_size = sum(size for _, size in task.input_files + task.output_files)
        total_size_gb = total_size / (1024 ** 3)
        execution_time = task.instructions * 6000 / device.mips
        storage_cost = total_size_gb * self.storage_cost_rate[device.dev_type] * execution_time
        exec_cost = (execution_time / 3600) * device.cost_per_hour
        return storage_cost + exec_cost

    def calculate_value(self, allocation):
        total_time = sum(self.estimate_time(task, device) for task, device in allocation)
        total_cost = sum(self.estimate_cost(task, device) for task, device in allocation)
        return -(total_time + total_cost)

    def task_allocation(self, tasks, fog_nodes, cloud_servers):
      unmatched_tasks = tasks[:]
      proposals = {task.id: [] for task in tasks}
      allocations = {}

      while unmatched_tasks:
          for task in unmatched_tasks[:]:
              prefs = sorted(fog_nodes + cloud_servers, key=lambda d: self.estimate_time(task, d))
              for device in prefs:
                  if device.id not in proposals[task.id]:
                      proposals[task.id].append(device.id)
                      allocations[task.id] = device  # Allocate task to device
                      unmatched_tasks.remove(task)
                      break
      # Return list of (task, device) pairs
      return [(task, allocations[task.id]) for task in tasks]

    def offload_location(self, tasks):
      F = [[(task, device)] for task in tasks for device in self.all_devices]
      if not F:
          return []

      # Initialize F0 with the best allocation
      F0 = max(F, key=lambda alloc: self.calculate_value(alloc))
      F.remove(F0)

      stop = False

      while not stop:

          stop = True
          visited = {tuple((t.id, d.id) for t, d in alloc): False for alloc in F}

          flag = True
          while flag:
              flag = False
              unvisited = [alloc for alloc in F if not visited[tuple((t.id, d.id) for t, d in alloc)]]
              if not unvisited:
                  break
              FLi = random.choice(unvisited)
              visited[tuple((t.id, d.id) for t, d in FLi)] = True

              tasks_to_allocate = [t for t, _ in FLi]
              allocated_pairs = self.task_allocation(tasks_to_allocate, self.fog_nodes, self.cloud_servers)
              candidate = allocated_pairs

              # We're minimizing cost+time, so "<" is correct
              if self.calculate_value(candidate) < self.calculate_value(F0):
                  F0 = candidate
                  F.remove(FLi)
                  flag = True

          if len(F0) <= 1:
              continue

          for partition in combinations(F0, len(F0) - 1):
              FO_ = list(partition)
              remaining = [item for item in F0 if item not in FO_]
              F_hat = [[item] for item in remaining]

              for idx, FLi in enumerate(F_hat):
                tasks_to_allocate = [t for t, _ in FLi]
                F_hat[idx] = self.task_allocation(tasks_to_allocate, self.fog_nodes, self.cloud_servers)

              if self.calculate_value(FO_ + sum(F_hat, [])) < self.calculate_value(F0):
                  F0 = FO_ + sum(F_hat, [])
                  stop = False
                  break

      return F0


    def get_device_by_id(self, dev_id):
        for device in self.all_devices:
            if device.id == dev_id:
                return device
        return None


# class NewSim



In [10]:
class  NewSim:
    def __init__(self, num_iot, num_fog, num_server, learning_rate=0.001, discount_factor=0.95,
                 exploration_rate=1.0, exploration_decay=0.995, exploration_min=0.01, batch_size=64, memory_size=2000, model_path=None):
        self.num_iot = num_iot
        self.num_fog = num_fog
        self.num_server = num_server
        self.num_totla_dev = num_iot + num_fog + num_server
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.exploration_min = exploration_min
        self.batch_size = batch_size
        self.memory_size = memory_size
        self.model_path = model_path
        self.env = simpy.Environment()
        self.reset()
        self.run()

    def reset(self):
        self.iot_devices = [Device(f'iot_{i}', 500, 0, self.env) for i in range(self.num_iot)]
        self.fog_devices = [Device(f'fog_{i}', 4000, 1/3600, self.env) for i in range(self.num_fog)]
        self.server_devices = [Device(f'cloud_{i}', 6000, 8/3600, self.env) for i in range(self.num_server)]
        self.agent = HeuristicAgent(self.iot_devices, self.fog_devices, self.server_devices)

        self.workflows = ensemble_of_workflows(name = 'CyberShake', size=100, distribution = 'constant', dax_path="/content/drive/My Drive/Zahra/dax/")
    def run(self):
        self.fog_env = FogEnv(self.env, self.iot_devices, self.fog_devices, self.server_devices,self.workflows)
        for workflow in self.workflows:
            self.env.process(process_workflow(self.env, workflow, self.fog_env, self.agent))

        self.env.run()  # Run simulation for a time period


# Data Collection


## The Config of Workflows

In [11]:
# Workflow configurations with specific sizes
workflow_configs = {
        'CyberShake': [30, 50, 100, 1000],
        'Montage': [20, 40, 60, 80, 100, 200, 300],
        'Inspiral': [30, 50, 100, 1000],
        'Sipht': [29, 58, 100, 968]
    }
workflow_distributions = ['constant', 'uniform']

In [12]:
def run_simulation_with_results_tracking(workflow_name, workflow_size, workflow_distribution, model_path):
    learning_rate = 0.001
    discount_factor = 0.99
    exploration_rate = 0.5
    exploration_decay = 0.995
    exploration_min = 0.05

    dax_path = "/content/drive/My Drive/Zahra/dax/"

    print(f"Running simulation for Workflow: {workflow_name}, Size: {workflow_size}, Distribution: {workflow_distribution}")

    # Set up the simulation with the current parameters
    simulation = NewSim(
        num_iot=10,
        num_fog=8,
        num_server=5,
        learning_rate=learning_rate,
        discount_factor=discount_factor,
        exploration_rate=exploration_rate,
        exploration_decay=exploration_decay,
        exploration_min=exploration_min,
        model_path = model_path
    )

    # Update the workflow parameters
    simulation.workflows = ensemble_of_workflows(
        name=workflow_name,
        size=workflow_size,
        distribution=workflow_distribution,
        dax_path=dax_path
    )

    # Run the simulation
    simulation.run()

    # Print results for current run
    print(f"Total cost for Workflow {workflow_name} ({workflow_distribution}, size={workflow_size}): {simulation.fog_env.cost}")
    print(f"Total time for Workflow {workflow_name} ({workflow_distribution}, size={workflow_size}): {simulation.env.now}")




# Results

## Cybershake - constant

In [13]:
for i in range(1, 11):
  run_simulation_with_results_tracking('CyberShake',10*i, 'constant', None)

Running simulation for Workflow: CyberShake, Size: 10, Distribution: constant
Total cost for Workflow CyberShake (constant, size=10): 29.815777777777797
Total time for Workflow CyberShake (constant, size=10): 147914.77999999982
Running simulation for Workflow: CyberShake, Size: 20, Distribution: constant
Total cost for Workflow CyberShake (constant, size=20): 60.765511111111195
Total time for Workflow CyberShake (constant, size=20): 162698.39999999988
Running simulation for Workflow: CyberShake, Size: 30, Distribution: constant
Total cost for Workflow CyberShake (constant, size=30): 88.9530444444442
Total time for Workflow CyberShake (constant, size=30): 175994.35999999993
Running simulation for Workflow: CyberShake, Size: 40, Distribution: constant
Total cost for Workflow CyberShake (constant, size=40): 119.81139999999944
Total time for Workflow CyberShake (constant, size=40): 188748.13999999966
Running simulation for Workflow: CyberShake, Size: 50, Distribution: constant
Total cost f

## Cybershake - uniform

In [43]:
for i in range(1, 11):
  run_simulation_with_results_tracking('CyberShake',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: CyberShake, Size: 10, Distribution: uniform
Total cost for Workflow CyberShake (uniform, size=10): 30.08757539252108
Total time for Workflow CyberShake (uniform, size=10): 143819.72999999986
Running simulation for Workflow: CyberShake, Size: 20, Distribution: uniform
Total cost for Workflow CyberShake (uniform, size=20): 79.78342036594611
Total time for Workflow CyberShake (uniform, size=20): 153094.1499999999
Running simulation for Workflow: CyberShake, Size: 30, Distribution: uniform
Total cost for Workflow CyberShake (uniform, size=30): 100.9082828225483
Total time for Workflow CyberShake (uniform, size=30): 157267.35
Running simulation for Workflow: CyberShake, Size: 40, Distribution: uniform
Total cost for Workflow CyberShake (uniform, size=40): 154.43850366949547
Total time for Workflow CyberShake (uniform, size=40): 171677.43999999994
Running simulation for Workflow: CyberShake, Size: 50, Distribution: uniform
Total cost for Workflow CyberShake (

## Montage - constant

In [44]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Montage',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Montage, Size: 10, Distribution: constant
Total cost for Workflow Montage (constant, size=10): 4.845283854438289
Total time for Workflow Montage (constant, size=10): 136805.54999999935
Running simulation for Workflow: Montage, Size: 20, Distribution: constant
Total cost for Workflow Montage (constant, size=20): 9.68389780455169
Total time for Workflow Montage (constant, size=20): 139603.04999999914
Running simulation for Workflow: Montage, Size: 30, Distribution: constant
Total cost for Workflow Montage (constant, size=30): 14.546897962215183
Total time for Workflow Montage (constant, size=30): 141627.40999999866
Running simulation for Workflow: Montage, Size: 40, Distribution: constant
Total cost for Workflow Montage (constant, size=40): 19.365106550162196
Total time for Workflow Montage (constant, size=40): 144584.61999999837
Running simulation for Workflow: Montage, Size: 50, Distribution: constant
Total cost for Workflow Montage (constant, size=50):

## Montage - uniform

In [45]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Montage',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Montage, Size: 10, Distribution: uniform
Total cost for Workflow Montage (uniform, size=10): 6.316836804777658
Total time for Workflow Montage (uniform, size=10): 138880.93999999974
Running simulation for Workflow: Montage, Size: 20, Distribution: uniform
Total cost for Workflow Montage (uniform, size=20): 7.234481602601078
Total time for Workflow Montage (uniform, size=20): 138666.35999999964
Running simulation for Workflow: Montage, Size: 30, Distribution: uniform
Total cost for Workflow Montage (uniform, size=30): 18.455376192120145
Total time for Workflow Montage (uniform, size=30): 143192.1299999998
Running simulation for Workflow: Montage, Size: 40, Distribution: uniform
Total cost for Workflow Montage (uniform, size=40): 26.580916509400378
Total time for Workflow Montage (uniform, size=40): 148165.55
Running simulation for Workflow: Montage, Size: 50, Distribution: uniform
Total cost for Workflow Montage (uniform, size=50): 28.603688452881865
Tot

## Sipht - constant

In [46]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Sipht',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Sipht, Size: 10, Distribution: constant
Total cost for Workflow Sipht (constant, size=10): 675.1622863251972
Total time for Workflow Sipht (constant, size=10): 437339.9668000001
Running simulation for Workflow: Sipht, Size: 20, Distribution: constant
Total cost for Workflow Sipht (constant, size=20): 1387.9733100412204
Total time for Workflow Sipht (constant, size=20): 757001.7352999994
Running simulation for Workflow: Sipht, Size: 30, Distribution: constant
Total cost for Workflow Sipht (constant, size=30): 1959.0462833555857
Total time for Workflow Sipht (constant, size=30): 1013637.9783999971
Running simulation for Workflow: Sipht, Size: 40, Distribution: constant
Total cost for Workflow Sipht (constant, size=40): 2552.418132631775
Total time for Workflow Sipht (constant, size=40): 1279292.9717000055
Running simulation for Workflow: Sipht, Size: 50, Distribution: constant
Total cost for Workflow Sipht (constant, size=50): 3146.757606903834
Total time

## Sipht - uniform

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Sipht',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

Running simulation for Workflow: Sipht, Size: 10, Distribution: uniform
Total cost for Workflow Sipht (uniform, size=10): 1104.4023287917225
Total time for Workflow Sipht (uniform, size=10): 628537.6382999978
Running simulation for Workflow: Sipht, Size: 20, Distribution: uniform
Total cost for Workflow Sipht (uniform, size=20): 1316.995990743965
Total time for Workflow Sipht (uniform, size=20): 726870.3016999974
Running simulation for Workflow: Sipht, Size: 30, Distribution: uniform
Total cost for Workflow Sipht (uniform, size=30): 4033.2158089019154
Total time for Workflow Sipht (uniform, size=30): 1942640.4872000103
Running simulation for Workflow: Sipht, Size: 40, Distribution: uniform
Total cost for Workflow Sipht (uniform, size=40): 3259.921761748836
Total time for Workflow Sipht (uniform, size=40): 1593885.5144999996
Running simulation for Workflow: Sipht, Size: 50, Distribution: uniform


## Inspiral - constant

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Inspiral',10*i, 'constant', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )

## Inspiral - uniform

In [ ]:
for i in range(1, 11):
  run_simulation_with_results_tracking('Inspiral',10*i, 'uniform', '/content/drive/My Drive/Zahra/Models/model_CyberShake_constant_10_cost.keras' )